In [215]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment, Font, Border, Side, PatternFill
from colorsys import hsv_to_rgb
from openpyxl.utils import get_column_letter

In [216]:
#Defining colors for filling
def color_generator(n):
    colors = []
    for i in range(n):
        hue = i/n
        saturation = 0.5 + (i%2)*0.25
        value = 0.8
        rgb = hsv_to_rgb(hue, saturation, value)
        hex_color = ''.join(f'{int(c*255):02X}' for c in rgb)
        colors.append(hex_color)
    return colors

In [217]:
#Defining importing files and exporting files
input_file = 'MEP.xlsx'
output_file = 'MEP_Out.xlsx'

In [218]:
#Importing data as Pandas DataFrame
df = pd.read_excel(input_file)
df.head()

,UID,FromRevit_GM,Plumbing,Fire,Sprinkler,Mechanical
0,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63ce0,NaN,MP-CS,FF-AA-FH-B2-01,SP-AA-GQH,AA-OHU-B2-01
1,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cea,NaN,MP-CS-KP,FF-AA-FH-01-01,SP-AA-DQU,AA-OHU-B2-02
2,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cef,PP-ARM-VP-1,MP-CR,FF-AA-FH-02-01,SP-AA-HFQR,AA-AHU-B2-01
3,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cf6,PP-W-VL-1,MP-CR-KP,FF-AA-FH-03-01,NaN,AA-AHU-B2-02
4,ed1ff2d1-bd87-4816-91c0-455213062d80-02d63cfa,PP-W-VL-1,MP-HS,NaN,NaN,AA-FCU-B2-01


In [219]:
#Finding unique elements from the specified columns by dropping duplicates
unique_elements = df['FromRevit_GM'].dropna().unique().tolist()

In [220]:
#Loading workbooka and worksheets
workbook = load_workbook(input_file)
work_sheet = workbook.active

In [221]:
#Generating colors 
colors = color_generator(len(unique_elements))
fills = [PatternFill(start_color = color, end_color=color, fill_type='solid') for color in colors]

In [222]:
#Finding elements to fill with respective colors according to unique elements
elements_to_fill = {unique_element: fills[idx%len(fills)] for idx, unique_element in enumerate(unique_elements)}

In [223]:
#Creating set for storing matching elements as set only holds the unique elements
matched_elements = set()

In [224]:
#Using logic for pattern-filling elements other than unique elements
for row in range(2, work_sheet.max_row+1):
    for col in range(3, work_sheet.max_column+1):
        cell = work_sheet.cell(row=row, column=col)
        if cell.value in elements_to_fill:
            matched_elements.add(cell.value)
            cell.fill = elements_to_fill[cell.value]

In [225]:
#Using logic for pattern-filling column that contains unique elements
for row in range(2, work_sheet.max_row+1):
    cell = work_sheet.cell(row=row, column=2)
    if cell.value in matched_elements:
        cell.fill = elements_to_fill[cell.value]
    else:
        cell.fill = PatternFill(fill_type=None)

In [226]:
#Listing unmatched elements
unmatched_elements = [element for element in unique_elements if element not in matched_elements]

In [227]:
#Creating a new column for appending unmatched elements
if unmatched_elements is not None:
    new_column = work_sheet.max_column+1
    work_sheet.cell(row=1, column=new_column, value='Unmatched Elements')
    for idx, values in enumerate(unmatched_elements, start=2):
        work_sheet.cell(row=idx, column=new_column, value=values)

In [228]:
'''
#Creating a new sheet for appending unmatched elements
if unmatched_elements is not None:
    new_sheet = workbook.create_sheet(title='UnmatchedElements')
    new_sheet = workbook['UnmatchedElements']
    new_sheet.cell(row=1, column=1, value='Unmatched Elements')
    for idx, values in enumerate(unmatched_elements, start=2):
        new_sheet.cell(row=idx, column=1, value=values)
        ''';

In [229]:
#Creating a new column for appending unmatched elements from excel based data
for col in range(3, work_sheet.max_column):
    not_found_elements = set()
    for row in range(2, work_sheet.max_row+1):
        cell = work_sheet.cell(row=row, column=col)
        if cell.value and cell.value not in matched_elements:
            not_found_elements.add(cell.value)

    new_column_idx = work_sheet.max_column+1
    original_column_header = work_sheet.cell(row=1, column=col).value
    work_sheet.cell(row=1, column=new_column_idx, value=original_column_header)
    for idx, values in enumerate(not_found_elements, start=2):
        work_sheet.cell(row=idx, column=new_column_idx, value=values)

In [230]:
#Adjusting the column width
for col in work_sheet.columns:
    max_length = 0
    column_letter = get_column_letter(col[0].column)
    for cell in col:
        if cell.value is not None:
            max_length = max(max_length, len(str(cell.value)))
    work_sheet.column_dimensions[column_letter].width = max_length+2

In [231]:
#Saving the result in new file
workbook.save(output_file)